In [54]:
import pickle
import numpy as np

import cv2

from json import load

In [55]:
MASK_THRESHOLD = 0.3
W = 3840
H = 2160

In [56]:
parking_config = load(open('pklot_config.json'))[0]

In [57]:
parking_config

{'annotations': [{'class': 'polygon',
   'id': 1,
   'xn': '212.61480052090155;327.6898214213895;152.33740862064596;39.4542928801673',
   'yn': '1358.9811992057626;1392.9557291859066;1645.0248225869755;1597.8988616467757'},
  {'class': 'polygon',
   'id': 2,
   'xn': '314.53839046133373;440.5729371618682;255.35695114108282;139.1859776605902',
   'yn': '1387.4759662858835;1424.7383540060414;1691.0548309471708;1641.7369648469617'},
  {'class': 'polygon',
   'id': 3,
   'xn': '436.1891268418496;574.2791519224352;381.39149784161725;250.97314082106422',
   'yn': '1426.9302591660507;1467.4805046262227;1746.9484125274078;1688.8629257871614'},
  {'class': 'polygon',
   'id': 4,
   'xn': '566.6965623783152;685.7582179920004;498.3976281503098;377.49005539532726',
   'yn': '1468.4270859021165;1509.960221581309;1791.462585629169;1738.853947102192'},
  {'class': 'polygon',
   'id': 5,
   'xn': '676.5286322855131;800.2050807524419;631.3036623237258;504.85833814485085',
   'yn': '1508.1143044400117;1

In [58]:
mask_data = pickle.load(open('masks.data', 'rb'))

In [59]:
boxes = mask_data['boxes']
masks = mask_data['masks']

In [60]:
for i in range(0, boxes.shape[2]):
    classID = int(boxes[0, 0, i, 1])
    if classID != 2:
        continue
    
    box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
    (startX, startY, endX, endY) = box.astype("int")
    boxW = endX - startX
    boxH = endY - startY

    # extract the pixel-wise segmentation for the object, resize
    # the mask such that it's the same dimensions of the bounding
    # box, and then finally threshold to create a *binary* mask
    mask = masks[i, classID]
    mask = cv2.resize(mask, (boxW, boxH),
                      interpolation=cv2.INTER_LINEAR)
    mask = (mask > MASK_THRESHOLD)